In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


+ ### read the srt file

In [28]:
import pandas as pd
import re

file_name = '/content/drive/My Drive/4th Year/DSC4013/day02/DJI_20230124113730_0001_W_Waypoint1.SRT'

frame_integer_re = '^[0-9]+$'

with open(file_name) as f :
    lines = [line.rstrip('\n') for line in f]
    
    
for line in lines[:22] :
    print(line)

1
00:00:00,000 --> 00:00:00,033
<font size="28">FrameCnt: 1, DiffTime: 33ms
2023-01-24 11:37:30.775
[iso: 110] [shutter: 1/400.0] [fnum: 2.8] [ev: 0] [focal_len: 24.00] [dzoom: 1.00] 
[latitude: 9.365171] [longitude: 80.089042] [rel_alt: 200.041 abs_alt: 123.051] 
[drone_speedx: -0.6 drone_speedy: -0.5 drone_speedz: 0.0] 
[drone_yaw: 127.1 drone_pitch: -1.5 drone_roll: -7.5] 
[gb_yaw: 128.3 gb_pitch: -25.0 gb_roll: 0.0] 

0
[ae_meter_md : 0] [dzoom_ratio: 10000, delta:0] [color_md : default] [ct : 5619] 
vsync[       0] target[0.0000 0.0000 0.0000 0.0000], shift[0.000000, 0.000000]</font>

2
00:00:00,033 --> 00:00:00,066
<font size="28">FrameCnt: 2, DiffTime: 33ms
2023-01-24 11:37:30.809
[iso: 110] [shutter: 1/400.0] [fnum: 2.8] [ev: 0] [focal_len: 24.00] [dzoom: 1.00] 
[latitude: 9.365171] [longitude: 80.089042] [rel_alt: 200.041 abs_alt: 123.051] 
[drone_speedx: -0.6 drone_speedy: -0.5 drone_speedz: 0.0] 
[drone_yaw: 127.1 drone_pitch: -1.5 drone_roll: -7.5] 


In [29]:
pattern_line_count = '^(1|[1-9]\d{0,3}|[1-2]\d{4}|290[0-1][0-9]|2902[0-2])$'
pattern_lng_lat_alt = r"\[(.*?)\]"

In [30]:
df = pd.DataFrame(columns=['Time Code', 'Frame Number', 'Latitude', 'Longitude', 'Altitude'])

+ ### create the dataframe with basic attributes

In [31]:
for index in range(len(lines)):
  line_str = lines[index]
  if re.search(pattern_line_count, line_str) != None:
    sequence_number = line_str
    start_end_time = lines[index+1]
    frame_count_str = lines[index+2]
    fc_sub = frame_count_str.split('>')[1]

    lng_lat_alt = lines[index+5]

    match = re.search(r'FrameCnt:\s*(\d+)', fc_sub)
    if match:
      fc = int(match.group(1))
      #print(fc)
    
    match = re.search(r'DiffTime:\s*(\d+)', fc_sub)
    if match:
      time_diff = int(match.group(1))
      #print(time_diff)

    

    #sub_parts = re.findall(pattern_lng_lat_alt, lng_lat_alt)
    sub_parts = lng_lat_alt.split('[')
    lat = sub_parts[1].split(":")[1].strip()[:-1]
    lng = sub_parts[2].split(":")[1].strip()[:-1]
    rel_alt = sub_parts[3].split(":")[1].strip()[:7]

    #lng = sub_parts.group(1)
    #print(lat)
    #print(lng)
    #print(rel_alt)


    #print(sequence_number)
    #print(start_end_time)
    #print(lng_lat_alt)
    #for subpart in sub_parts:
    #  print(subpart.group(1))

    entry = {
        'Time Code': start_end_time,
        'Frame Number': fc,
        'Latitude':lat,
        'Longitude':lng,
        'Altitude':rel_alt
    }

    df.loc[len(df)] = entry 

In [32]:
df.head(10)

,Time Code,Frame Number,Latitude,Longitude,Altitude
0,"00:00:00,000 --> 00:00:00,033",1,9.365171,80.089042,200.041
1,"00:00:00,033 --> 00:00:00,066",2,9.365171,80.089042,200.041
2,"00:00:00,066 --> 00:00:00,100",3,9.365170,80.089041,200.032
3,"00:00:00,100 --> 00:00:00,133",4,9.365170,80.089041,200.032
4,"00:00:00,133 --> 00:00:00,166",5,9.365170,80.089041,200.032
5,"00:00:00,166 --> 00:00:00,200",6,9.365170,80.089040,200.049
6,"00:00:00,200 --> 00:00:00,233",7,9.365170,80.089040,200.049
7,"00:00:00,233 --> 00:00:00,266",8,9.365170,80.089040,200.049
8,"00:00:00,266 --> 00:00:00,300",9,9.365169,80.089040,200.040
9,"00:00:00,300 --> 00:00:00,333",10,9.365169,80.089040,200.040


In [33]:
df.shape

(29022, 5)

+ ### calculate the distance and modify the dataframe

In [34]:
import math

df['Latitude'] = df['Latitude'].astype(float)
df['Longitude'] = df['Longitude'].astype(float)

def distance(lat1, lon1, lat2, lon2):
    R = 6371 # radius of Earth in km
    dLat = math.radians(lat2 - lat1)
    dLon = math.radians(lon2 - lon1)
    a = math.sin(dLat / 2) * math.sin(dLat / 2) + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dLon / 2) * math.sin(dLon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = R * c
    return d

df['Distance'] = df.apply(lambda x: distance(x['Latitude'], x['Longitude'], df.iloc[0]['Latitude'], df.iloc[0]['Longitude']), axis=1)

In [35]:
df.shape

(29022, 6)

In [36]:
df.head(10)

,Time Code,Frame Number,Latitude,Longitude,Altitude,Distance
0,"00:00:00,000 --> 00:00:00,033",1,9.365171,80.089042,200.041,0.000000
1,"00:00:00,033 --> 00:00:00,066",2,9.365171,80.089042,200.041,0.000000
2,"00:00:00,066 --> 00:00:00,100",3,9.365170,80.089041,200.032,0.000156
3,"00:00:00,100 --> 00:00:00,133",4,9.365170,80.089041,200.032,0.000156
4,"00:00:00,133 --> 00:00:00,166",5,9.365170,80.089041,200.032,0.000156
5,"00:00:00,166 --> 00:00:00,200",6,9.365170,80.089040,200.049,0.000246
6,"00:00:00,200 --> 00:00:00,233",7,9.365170,80.089040,200.049,0.000246
7,"00:00:00,233 --> 00:00:00,266",8,9.365170,80.089040,200.049,0.000246
8,"00:00:00,266 --> 00:00:00,300",9,9.365169,80.089040,200.040,0.000312
9,"00:00:00,300 --> 00:00:00,333",10,9.365169,80.089040,200.040,0.000312


In [37]:
df.tail()

,Time Code,Frame Number,Latitude,Longitude,Altitude,Distance
29017,"00:16:08,200 --> 00:16:08,234",29018,9.307987,80.175756,200.006,11.443584
29018,"00:16:08,234 --> 00:16:08,267",29019,9.307987,80.175756,200.006,11.443584
29019,"00:16:08,267 --> 00:16:08,300",29020,9.307983,80.175766,199.998,11.444744
29020,"00:16:08,300 --> 00:16:08,334",29021,9.307983,80.175766,199.998,11.444744
29021,"00:16:08,334 --> 00:16:08,367",29022,9.307983,80.175766,199.998,11.444744


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29022 entries, 0 to 29021
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Time Code     29022 non-null  object 
 1   Frame Number  29022 non-null  int64  
 2   Latitude      29022 non-null  float64
 3   Longitude     29022 non-null  float64
 4   Altitude      29022 non-null  object 
 5   Distance      29022 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 1.5+ MB


+ ### build a function to calculate the speed

In [39]:
def speed(Time_Code, distance, start_time):
    const = 3600000

    end_time = Time_Code.split('-->')[1].strip()
    end_time = end_time.replace(':', '').replace(',', '')
    end_int = int(end_time)

    time_diff = end_int - start_time
    start_time = end_int

    print(f"end_time: {end_time}, distance: {distance}, time_diff:{time_diff}")

    speed = (distance * const) / time_diff

    print(f"speed: {speed}")

    return speed


+ ### iteratively read the rows and add speed column to the dataframe

In [40]:
start_time = None
for i, row in df.iterrows():
    if start_time is None:
        start_time = int(row['Time Code'].split('-->')[0].replace(':', '').replace(',', ''))
        df.loc[i, 'Speed'] = 0
    else:
        df.loc[i, 'Speed'] = speed(row['Time Code'], row['Distance'], start_time)


Streaming output truncated to the last 5000 lines.
end_time: 001444984, distance: 10.484513726350846, time_diff:1444984
speed: 26.120877058059495
end_time: 001445017, distance: 10.484513726350846, time_diff:1445017
speed: 26.120280532937013
end_time: 001445050, distance: 10.484513726350846, time_diff:1445050
speed: 26.119684035059716
end_time: 001445084, distance: 10.485756434989923, time_diff:1445084
speed: 26.122165331540398
end_time: 001445117, distance: 10.485756434989923, time_diff:1445117
speed: 26.12156881827819
end_time: 001445151, distance: 10.485756434989923, time_diff:1445151
speed: 26.12095425735008
end_time: 001445184, distance: 10.486845394324664, time_diff:1445184
speed: 26.123070432255542
end_time: 001445217, distance: 10.486845394324664, time_diff:1445217
speed: 26.122473939601313
end_time: 001445251, distance: 10.486845394324664, time_diff:1445251
speed: 26.12185939990271
end_time: 001445284, distance: 10.488088124894356, time_diff:1445284
speed: 26.124358430329043
en

In [41]:
df.head()

,Time Code,Frame Number,Latitude,Longitude,Altitude,Distance,Speed
0,"00:00:00,000 --> 00:00:00,033",1,9.365171,80.089042,200.041,0.000000,0.000000
1,"00:00:00,033 --> 00:00:00,066",2,9.365171,80.089042,200.041,0.000000,0.000000
2,"00:00:00,066 --> 00:00:00,100",3,9.365170,80.089041,200.032,0.000156,5.623520
3,"00:00:00,100 --> 00:00:00,133",4,9.365170,80.089041,200.032,0.000156,4.228211
4,"00:00:00,133 --> 00:00:00,166",5,9.365170,80.089041,200.032,0.000156,3.387663


In [42]:
df.tail()

,Time Code,Frame Number,Latitude,Longitude,Altitude,Distance,Speed
29017,"00:16:08,200 --> 00:16:08,234",29018,9.307987,80.175756,200.006,11.443584,25.616237
29018,"00:16:08,234 --> 00:16:08,267",29019,9.307987,80.175756,200.006,11.443584,25.615711
29019,"00:16:08,267 --> 00:16:08,300",29020,9.307983,80.175766,199.998,11.444744,25.617781
29020,"00:16:08,300 --> 00:16:08,334",29021,9.307983,80.175766,199.998,11.444744,25.617239
29021,"00:16:08,334 --> 00:16:08,367",29022,9.307983,80.175766,199.998,11.444744,25.616714


+ ### export the dataframe as csv and json file

In [43]:
df.to_csv('DJI.csv', index=False)

In [44]:
df.to_json('DJI.json', orient='records')